In [2]:
import pandas as pd
from sklearn import preprocessing
from datetime import datetime
from sklearn.svm import SVR
import numpy as np

In [3]:
df = pd.read_csv("Data/ticket.csv", delimiter="\t")

#### Removing redundant sections and rows with NaN for CheckinTime

In [7]:
df.drop("TIME SECTION",1, inplace = True)
df.drop("TICKET",1, inplace = True)
df.drop("TICKET NUM",1, inplace = True)
df.drop("TABLE TYPE",1, inplace = True)
df.drop("STATUS",1, inplace = True)
df.drop("CHANNEL",1, inplace = True)
df = df.dropna()

In [9]:
df.replace(to_replace=113442, value=0, inplace=True)
df.replace(to_replace=113558, value=0, inplace=True)
df.replace(to_replace=113574, value=0, inplace=True)
df.replace(to_replace=136888, value=3, inplace=True)

In [14]:
df.head(5)

,REST_ID,TABLE SIZE,TIME (SEC),MONTH,DAY,WAIT TIME
0,0,5,40159,1,5,442
1,0,4,40545,1,5,55
2,0,2,40631,1,5,10
3,0,2,41681,1,5,12
4,0,5,42660,1,5,576


#### Parsing the DateTime and adding columns based on it

In [11]:
dater = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S.%f")
dater2 = lambda c: datetime.strptime(c, "%Y-%m-%d %H:%M:%S")

In [12]:
df["TIME CHECKIN"] = df["CHECKINTIME"].apply(lambda a: dater2(a).hour*3600 + dater2(a).minute*60 + dater2(a).second)
df["TIME (SEC)"] = df["CREATE TIME"].apply(lambda a: dater(a).hour*3600 + dater(a).minute*60 + dater(a).second)
df["MONTH"] = df["CREATE TIME"].apply(lambda a: int(a.split()[0].split('-')[1]))
df["DAY"] = df["CREATE TIME"].apply(lambda a: dater(a).isoweekday())
df["WAIT TIME"] = df["TIME CHECKIN"] - df["TIME (SEC)"]

In [13]:
df.drop("CREATE TIME",1, inplace = True)
df.drop("CHECKINTIME",1, inplace = True)
df.drop("TIME CHECKIN",1, inplace = True)

#### Outputting to a csv

In [15]:
df.to_csv("Data/cleaned.csv", index=False)